In [80]:
import onnx

onnx_model = onnx.load('weights/yolo-fastest.onnx')

In [46]:
graph = onnx_model.graph
node  = graph.node

In [47]:
for i in range(len(node)):
	print(node[i])    

 "module_list.0.BatchNorm2d.bias"
input: "module_list.0.BatchNorm2d.running_mean"
input: "module_list.0.BatchNorm2d.running_var"
output: "504"
name: "BatchNormalization_1"
op_type: "BatchNormalization"
attribute {
  name: "epsilon"
  f: 9.999999747378752e-05
  type: FLOAT
}
attribute {
  name: "momentum"
  f: 0.9700000286102295
  type: FLOAT
}

input: "504"
output: "505"
name: "LeakyRelu_2"
op_type: "LeakyRelu"
attribute {
  name: "alpha"
  f: 0.10000000149011612
  type: FLOAT
}

input: "505"
input: "module_list.1.Conv2d.weight"
output: "506"
name: "Conv_3"
op_type: "Conv"
attribute {
  name: "dilations"
  ints: 1
  ints: 1
  type: INTS
}
attribute {
  name: "group"
  i: 1
  type: INT
}
attribute {
  name: "kernel_shape"
  ints: 1
  ints: 1
  type: INTS
}
attribute {
  name: "pads"
  ints: 0
  ints: 0
  ints: 0
  ints: 0
  type: INTS
}
attribute {
  name: "strides"
  ints: 1
  ints: 1
  type: INTS
}

input: "506"
input: "module_list.1.BatchNorm2d.weight"
input: "module_list.1.BatchNorm

In [60]:
for i in range(len(node)):
    
    if node[i].output == '1469':
        node_rise = node[i]
        print(i)
        print(node_rise)

In [25]:
for node in graph.node:
    for at in node.attribute:
        if at.type == 7:
            print('#'*50)
            print(node)
            at.type = 6    

##################################################
Conv_0
##################################################
Conv_0
##################################################
Conv_0
##################################################
Conv_0
##################################################
Conv_3
##################################################
Conv_3
##################################################
Conv_3
##################################################
Conv_3
##################################################
Conv_6
##################################################
Conv_6
##################################################
Conv_6
##################################################
Conv_6
##################################################
Conv_9
##################################################
Conv_9
##################################################
Conv_9
##################################################
Conv_9
##################################################
Conv_11
#############

In [114]:
onnx.save(onnx_model, 'weights/yolo-fastest-transfer.onnx')

In [49]:
for i in range(len(node)):
    for tensor_info in [graph.node[i]]:
        print(tensor_info.name)
        att = tensor_info.attribute
        for at in att:
            print('get', getattr(at,'type',None))
            if (getattr(at, "type", None) == 7):
                print('get INT64!!!!!!!!!!!!!!!!!!')
                setattr(at, "type", 6)


Conv_0
get 7
get INT64!!!!!!!!!!!!!!!!!!
get 2
get 7
get INT64!!!!!!!!!!!!!!!!!!
get 7
get INT64!!!!!!!!!!!!!!!!!!
get 7
get INT64!!!!!!!!!!!!!!!!!!
BatchNormalization_1
get 1
get 1
LeakyRelu_2
get 1
Conv_3
get 7
get INT64!!!!!!!!!!!!!!!!!!
get 2
get 7
get INT64!!!!!!!!!!!!!!!!!!
get 7
get INT64!!!!!!!!!!!!!!!!!!
get 7
get INT64!!!!!!!!!!!!!!!!!!
BatchNormalization_4
get 1
get 1
LeakyRelu_5
get 1
Conv_6
get 7
get INT64!!!!!!!!!!!!!!!!!!
get 2
get 7
get INT64!!!!!!!!!!!!!!!!!!
get 7
get INT64!!!!!!!!!!!!!!!!!!
get 7
get INT64!!!!!!!!!!!!!!!!!!
BatchNormalization_7
get 1
get 1
LeakyRelu_8
get 1
Conv_9
get 7
get INT64!!!!!!!!!!!!!!!!!!
get 2
get 7
get INT64!!!!!!!!!!!!!!!!!!
get 7
get INT64!!!!!!!!!!!!!!!!!!
get 7
get INT64!!!!!!!!!!!!!!!!!!
BatchNormalization_10
get 1
get 1
Conv_11
get 7
get INT64!!!!!!!!!!!!!!!!!!
get 2
get 7
get INT64!!!!!!!!!!!!!!!!!!
get 7
get INT64!!!!!!!!!!!!!!!!!!
get 7
get INT64!!!!!!!!!!!!!!!!!!
BatchNormalization_12
get 1
get 1
LeakyRelu_13
get 1
Conv_14
get 7


In [55]:
for n in graph.node:
    if n.name == 'Concat_600':
        for i in n.input:
            print(i.type)

AttributeError: 'str' object has no attribute 'type'

In [56]:
len(node)

995

In [ ]:

graph = onnx_model.graph
for tensor_info in [graph.node[0]]:
    print(tensor_info.attribute)
    att= tensor_info.attribute
    for at in att:
        print('get',getattr(at, "type", None))

    if (getattr(at, "type", None)==2):
        setattr(at, "type",2)
    
    setattr(at, "type",7)
    print('############',tensor_info.attribute)    
    tp=getattr(graph.input[0], "type", None)
    if getattr(tp, "tensor_type", None).elem_type==2:
        getattr(tp, "tensor_type", None).elem_type=6


In [64]:
#获取节点和节点的输入输出名列表，一般节点的输入将来自于上一层的输出放在列表前面，参数放在列表后面
def getNodeAndIOname(nodename,model):
    for i in range(len(model.graph.node)):
        if model.graph.node[i].name == nodename:
            Node = model.graph.node[i]
            input_name = model.graph.node[i].input
            output_name = model.graph.node[i].output
    return Node,input_name,output_name

#获取对应输入信息
def getInputTensorValueInfo(input_name,model):
    in_tvi = []
    for name in input_name:
        for params_input in model.graph.input:
            if params_input.name == name:
               in_tvi.append(params_input)
        for inner_output in model.graph.value_info:
            if inner_output.name == name:
                in_tvi.append(inner_output)
    return in_tvi

#获取对应输出信息
def getOutputTensorValueInfo(output_name,model):
    out_tvi = []
    for name in output_name:
        out_tvi = [inner_output for inner_output in model.graph.value_info if inner_output.name == name]
        if name == model.graph.output[0].name:
            out_tvi.append(model.graph.output[0])
    return out_tvi

#获取对应超参数值
def getInitTensorValue(input_name,model):
    init_t = []
    for name in input_name:
        init_t = [init for init in model.graph.initializer if init.name == name]
    return init_t


In [111]:
onnx_model = onnx.load('weights/yolo-fastest.onnx')


In [96]:
for n in graph.node:
    print(n.input_tensors)

AttributeError: 'NodeProto' object has no attribute 'input_tensors'

In [101]:
weights = onnx_model.graph.initializer
from onnx import numpy_helper
w1 = numpy_helper.to_array(weights[0])

In [113]:
for initializer in onnx_model.graph.initializer:
    if initializer.data_type == 7:
        initializer.data_type = 6

In [112]:
for initializer in onnx_model.graph.initializer:
    if initializer.data_type == 7:
        print(initializer.name)

1536
1537
1538
1539
1540
1541
1542
1543
1544
1550
1551
1552
1553
1554
1555
1556
1557
1558
